## How to use the AWSLambda executor in a workflow

Covalent support execution of electrons/entire lattices on services provided by different cloud providers. In this guide we will illustrate how to use a simple cloud executor for dispatching a electron for remote execution. In this example we will use the AWS Lambda executor to executor an electron using the AWS Lambda service.

We first give a quick overview of how the executor executes the electron using AWS Lambda behind the scenes here. Details regarding the executor's implementation can be viewed [here](https://github.com/AgnostiqHQ/covalent-awslambda-plugin).


### Executor overview and configuration

The `AWSLambdaExecutor` takes a few input arguments that are needed to successfully execute a task using a Lambda function on AWS on behalf of the user. The input arguments to the executor are the following

* `credentials`: Path to the AWS credentials file (default: `~/.aws/credentials`)
* `profile`: The AWS user profile to be used to interact with the AWS services (default: `default`)
* `region`: AWS region (default: `us-east-1`)
* `s3_bucket_name`: Name of the S3 bucket to be used to cache temporary files generated during execution (default: `covalent-lambda-job-resources`)
* `role_name`: AWS IAM role name to be used to provision the Lambda function with (default: `CovalentLambdaExecutionRole`)
* `cache_dir`: Path to the cache directory on the local filesystem (default: `~/.cache/covalent`)
* `poll_freq`: Time interval between successive polls to the lambda function (default: `5`)
* `timeout`: Duration in seconds before the Lambda function times out (default: `60`, max: `900`)
* `cleanup`: Boolean flag representing whether or not to cleanup the temporary files that get generated during execution (default: `True`)

Users can also provide their configuration values in the covalent configuration file (`~/.config/covalent/covalent.conf`) as follows

```
# ~/.config/covalent/covalent.conf
[executors.awslambda]
credentials = "/home/<user>/.aws/credentials"
profile = "default"
region = "us-east-1"
lambda_role_name = "CovalentLambdaExecutionRole"
s3_bucket_name = "covalent-lambda-job-resources"
cache_dir = "/home/<user>/.cache/covalent"
poll_freq = 5
timeout = 60
memory_size = 512
cleanup = true
```

Since there are default values associated with each of these input arguments and if the configuration values have been provided in `covalent.conf`, the executor can be simpliy instantiated without passing any input arguments (assuming the necessary AWS resources, IAM roles, policies with the matching names have already been provisioned/configured)

```python
from covalent.executor import AWSLambdaExecutor

awslambda = AWSLambdaExecutor()

@ct.electron(executor=awslambda)
def task(..):
    ...
```

Alternatively, users can also provide the executor's module name i.e. `awslambda` as the executor of the electron. This is a convenient shortcut that can be used in Covalent when all the executor specific configuration options (i.e. arguments to its constructor) have been specified in Covalent's configuration file (`~/.config/covalent/covalent.conf`)

```python
from covalent.executor import AWSLambdaExecutor

@ct.electron(executor="awslambda")
def task(...):
    ...
```

#### Cloud permissions and policies

Since this executor interacts with different cloud resources aka. AWS S3 and AWS Lambda, the permissions associated with the AWS profile must be configured properly. For the executor to operate smoothly, the following IAM policy document can be used

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:*",
                "s3-object-lambda:*"
            ],
            "Resource": [
                "arn:aws:s3:::<bucket-name>",
                "arn:aws:s3:::<bucket-name>/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "cloudformation:DescribeStacks",
                "cloudformation:ListStackResources",
                "cloudwatch:ListMetrics",
                "cloudwatch:GetMetricData",
                "ec2:DescribeSecurityGroups",
                "ec2:DescribeSubnets",
                "ec2:DescribeVpcs",
                "kms:ListAliases",
                "iam:GetPolicy",
                "iam:GetPolicyVersion",
                "iam:GetRole",
                "iam:GetRolePolicy",
                "iam:ListAttachedRolePolicies",
                "iam:ListRolePolicies",
                "iam:ListRoles",
                "lambda:*",
                "logs:DescribeLogGroups",
                "states:DescribeStateMachine",
                "states:ListStateMachines",
                "tag:GetResources",
                "xray:GetTraceSummaries",
                "xray:BatchGetTraces"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "lambda.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogStreams",
                "logs:GetLogEvents",
                "logs:FilterLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/lambda/*"
        }
    ]
}
```

Here `<bucket-name>` is the S3 bucket that the executor will use to store/cache temporary files and objects. Secondly the lambda function created by this executor to execute the electron's code also needs certain permissions in order to interact with AWS sevices. To this end, the following IAM policy document can be used

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:*"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject"
            ],
            "Resource": "arn:aws:s3:::*"
        }
    ]
}
```

### Executor installation

The lambda executor plugin can be installed either via `pypi` by running `pip install covalent-awslambda-plugin`. Alternatively, the github repo with the source code can be cloned as follows

```sh
git clone https://github.com/AgnostiqHQ/covalent-awslambda-plugin.git
cd covalent-awslambda-plugin
pip install -e .
```


### Workflow example

We now provide a simple example on how to use this executor in a covalent workflow. We use a trivally simple example where we dispatch the `identity` electron to be executed as a lambda function in AWS, since the purpose of the example is to demonstrate the use of the lambda executor.


In [ ]:
# Import covalent and the executor
import covalent as ct
from covalent.executor import AWSLambdaExecutor

In [ ]:
# Create an instance of the lambda executor with configuration values provided inline

awslambda = AWSLambdaExecutor(credentials="~/.aws/credentials",
            profile="default",
            region="us-east-1",
            lambda_role_name="CovalentLambdaExecutionRole",
            s3_bucket_name="covalent-lambda-job-resources",
            cache_dir="/home/<user>/.cache/covalent",
            poll_freq=5,
            timeout = 60,
            memory_size = 512,
            cleanup = True)

In [ ]:
# Create the electron and the workflow
@ct.electron(executor=awslambda)
def identity(x):
    return x

@ct.lattice
def workflow(x):
    return identity(x)


dispatch_id = ct.dispatch(workflow)(1)
result = ct.get_result(dispatch_id, wait=True)

print(result)